# Day 4: Giant Squid

[https://adventofcode.com/2021/day/4](https://adventofcode.com/2021/day/4)

## Description

### Part One

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you _can_ see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play [bingo](https://en.wikipedia.org/wiki/Bingo_(American_version))?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is _marked_ on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board _wins_. (Diagonals don't count.)

The submarine has a _bingo subsystem_ to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

    7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

    22 13 17 11  0
     8  2 23  4 24
    21  9 14 16  7
     6 10  3 18  5
     1 12 20 15 19

     3 15  0  2 22
     9 18 13 17  5
    19  8  7 25 23
    20 11 10 24  4
    14 21 16 12  6

    14 21 17 24  4
    10 16 15  9 19
    18  8 23 26 20
    22 11 13  6  5
     2  0 12  3  7


After the first five numbers are drawn (`7`, `4`, `9`, `5`, and `11`), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

    22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
     8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
    21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
     6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
     1 12 20 15 19        14 21 16 12  6         2  0 12  3  7


After the next six numbers are drawn (`17`, `23`, `2`, `0`, `14`, and `21`), there are still no winners:

    22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
     8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
    21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
     6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
     1 12 20 15 19        14 21 16 12  6         2  0 12  3  7


Finally, `24` is drawn:

    22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
     8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
    21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
     6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
     1 12 20 15 19        14 21 16 12  6         2  0 12  3  7


At this point, the third board _wins_ because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: _`14 21 17 24 4`_).

The _score_ of the winning board can now be calculated. Start by finding the _sum of all unmarked numbers_ on that board; in this case, the sum is `188`. Then, multiply that sum by _the number that was just called_ when the board won, `24`, to get the final score, `188 * 24 = _4512_`.

To guarantee victory against the giant squid, figure out which board will win first. _What will your final score be if you choose that board?_

### Part Two

On the other hand, it might be wise to try a different strategy: <span title="That's 'cuz a submarine don't pull things' antennas out of their sockets when they lose. Giant squid are known to do that.">let the giant squid win</span>.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to _figure out which board will win last_ and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after `13` is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to `148` for a final score of `148 * 13 = _1924_`.

Figure out which board will win last. _Once it wins, what would its final score be?_

In [1]:
BOARD_SIZE = 5
MARKED = None
WON = None


def get_drawn_numbers():
    with open('input') as input_file:
        return list(map(int, input_file.readline().split(',')))


def get_boards():
    boards = []
    current_board = []
    first_line_skipped = False

    with open('input') as input_file:
        for line in input_file:
            line = line.strip()

            if not line or not first_line_skipped:
                first_line_skipped = True
                continue

            current_board.append(list(map(int, line.split())))
            if len(current_board) == BOARD_SIZE:
                boards.append(current_board)
                current_board = []

    return boards


def mark_board(board, drawn_number):
    for row in board:
        for cell_no in range(BOARD_SIZE):
            if row[cell_no] == drawn_number:
                row[cell_no] = MARKED


def is_board_won(board):
    def _check_board(board):
        for row in board:
            if all(cell is MARKED for cell in row):
                return True
        return False

    if _check_board(board):
        return True

    transposed_board = [[board[j][i] for j in range(BOARD_SIZE)] for i in range(BOARD_SIZE)]

    return _check_board(transposed_board)


def get_board_sum(board):
    return sum(sum(cell for cell in row if cell is not MARKED) for row in board)


def get_results(drawn_numbers, boards):
    results = []

    for drawn_number in drawn_numbers:
        for board_no in range(len(boards)):
            board = boards[board_no]
            if board is WON:
                continue

            mark_board(board, drawn_number)

            if is_board_won(board):
                board_sum = get_board_sum(board)
                results.append((board_no, drawn_number * board_sum))
                boards[board_no] = WON

    return results


drawn_numbers = get_drawn_numbers()
boards = get_boards()
results = get_results(drawn_numbers, boards)

first_to_win = results[0]
last_to_win = results[-1]

print(f'First winning board is {first_to_win[0]} with sum {first_to_win[1]}')
print(f'Last winning board is {last_to_win[0]} with sum {last_to_win[1]}')

First winning board is 41 with sum 46920
Last winning board is 29 with sum 12635
